In [ ]:
import yaml
from rdkit import Chem
from rdkit.Chem.Descriptors import MolWt
from MDSetup.forcefield import forcefield
from MDSetup.tools.systemsetup import (
    get_system_volume,
    generate_initial_configuration
)

In [30]:
FOLDER_PRECISION = 2

class test():

    def __init__(self,system_setup) -> None:
        self.system_setup = system_setup
        self.state_folder = "_".join([ 
                    f"{folder_attribute[:4]}_%.{FOLDER_PRECISION}f"
                    for folder_attribute in self.system_setup['folder_attributes']
                ])
        
a = test(system_setup={"folder_attributes": ["temperature", "pressure", "density", "fraction"]})

local_vars = {"temperature":300,"pressure":10,"density":500,"fraction":0.5}

sub_txt = ' '.join( 
                (f"{folder_attribute}: {local_vars[folder_attribute]:.{FOLDER_PRECISION}f} "
                 f"{'K' if folder_attribute=='temperature' else 'bar' if folder_attribute=='pressure' else 'kg/m^3' if folder_attribute=='density' else 'mol/mol'}"
                 ", ") for
                 folder_attribute in a.system_setup['folder_attributes']
            )

sub_txt

'temperature: 300.00 K,  pressure: 10.00 bar,  density: 500.00 kg/m^3,  fraction: 0.50 mol/mol, '

(0, 1, 2)

In [19]:
a.state_folder

'temp_%.2f_pres_%.2f_dens_%.2f_frac_%.2f'

In [16]:
a.__dict__

{'system_setup': {'folder_attributes': ['temperature',
   'pressure',
   'density',
   'fraction']},
 'state_folder': 'temp_%.2f_pres_%.2f_dens_%.2f_frac_%.2f'}

'temp_%.2f_pres_%.2f_dens_%.2f_frac_%.2f'

# System building for LAMMPS and GROMACS

This notebook shows the capibility of the MDSetup forcefield class to map the predefined force field format to arbitrary SMILES given molecules.

In [ ]:
# Define molecules via SMILES
smiles = [
    "CCC",
    "C=C"
]

# Define self given names for each molecule
residues = [
    "propane",
    "ethene"
]

# Define system conditions

# Molecule numbers of each species
molecule_numbers = [ 100, 300 ]

# Get molar masses from rdkit
molar_masses = [
    MolWt(Chem.MolFromSmiles(smile)) for smile in smiles
]

# Define density
density = 600

## Build system for LAMMPS

### Write molecule and topology files

In [ ]:
# Read in LAMMPS forcefield files
forcefield_paths = [ 
    "../../force-fields/TAMie_hydrocarbons_lammps.toml" 
]

# Define building folder
building_folder = "lammps"

# Initialize force field class
ff = forcefield( 
    smiles = smiles,
    force_field_paths = forcefield_paths,
    verbose=True
)

# Define path to molecule template file
molecule_template = "../../templates/lammps/template_lammps.mol"

# Write molecule files in LAMMPS compatible format
ff.write_molecule_files(
    molecule_template = molecule_template, 
    molecule_path = building_folder,
    residues = residues
)

# Define path to the topology (force field) file
topology_template = "../../templates/lammps/template_lammps.params"

# Read in default settings for force field file
default_ff = yaml.safe_load(open("lammps/defaults.yaml"))

# Write topology (force field) file
ff.write_topology_file(
    topology_template = topology_template, 
    topology_path = building_folder, 
    system_name = "propane_ethene",
    **default_ff["nonbonded"]
)

print(f"\n\nMolecule files:\n {', '.join(ff.molecule_files)}\n")
print(f"\nTopology file:\n {ff.topology_file}")

### Write lammps input for building the system

In [ ]:
# Define box type
box_setup = dict(
  box_type = "cubic",
  z_x_relation = 1.0,
  z_y_relation = 1.0
)

# Define unit conversion if unit is not Angstrom
distance_conversion = 1

# Get box dimensions 
box_dim = get_system_volume(
    molar_masses=molar_masses,
    molecule_numbers=molecule_numbers,
    density=density,
    unit_conversion=distance_conversion,
    **box_setup,
)

# Define bash template to run building
build_template = "../../templates/bash/build_system_lmp_pbs.sh"

# Define lammps building template
building_template = "../../templates/lammps/template_lammps.build"

# Define molecule list 
molecules_list = [
    { "name": res,
      "number": num 
    } for res,num in zip(residues,molecule_numbers)
]

# Generate initial configuration (needs LAMMPS on the machine)
generate_initial_configuration(
    destination_folder = building_folder,
    build_template = build_template,
    software = ff.software,
    coordinate_paths = ff.molecule_files,
    molecules_list = molecules_list,
    box = box_dim,
    build_input_template = building_template,
    force_field_file = ff.topology_file
)

## Build system for GROMACS